### Task 2

In [11]:
import logging
import pandas as pd
import os
import sys

sys.path.append(os.path.abspath('../scripts'))

from forecasting.arima_model import train_arima
from forecasting.sarima_model import train_sarima
from forecasting.lstm_model import preprocess_data, build_lstm, train_lstm
from forecasting.evaluation_metrics import evaluate_forecast
from utils.mlflow_utils import (
    start_mlflow_experiment, log_params, log_metrics, log_model, end_mlflow_experiment
)
import matplotlib.pyplot as plt

In [14]:
from utils.logging_config import setup_logger
setup_logger(log_file="task2_mlflow_forecasting.log", log_level=logging.INFO)

2024-11-17 14:30:58,543 - INFO - Logger initialized. Log file: logs\task2_mlflow_forecasting.log


In [15]:
logging.info("Task 2: Forecasting Pipeline with Multiple Datasets Started.")

2024-11-17 14:31:12,986 - INFO - Task 2: Forecasting Pipeline with Multiple Datasets Started.


In [16]:
datasets = {
    "TSLA": pd.read_csv("../data/processed/TSLA_processed.csv", parse_dates=["Date"], index_col="Date"),
    "SPY": pd.read_csv("../data/processed/SPY_processed.csv", parse_dates=["Date"], index_col="Date"),
    "BND": pd.read_csv("../data/processed/BND_processed.csv", parse_dates=["Date"], index_col="Date"),
}
